In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
df = pd.read_csv ('SCORE_csv.csv')
df['H_index'] = ""
df['SJR_score'] = ""
df = df[['publication_CR','H_index','SJR_score']].drop_duplicates(subset='publication_CR', keep="last")
divsList = []

for name in df['publication_CR']:
  if requests.get('https://www.scimagojr.com/journalsearch.php?q='+name.replace(" ", "+")).status_code==200: 
      divs = BeautifulSoup(requests.get('https://www.scimagojr.com/journalsearch.php?q='+name.replace(" ", "+")).content, 'html.parser').select("div.journaldescription.colblock div.search_results a")
      divsList.append(divs)

In [3]:
divsList = list(filter(None, divsList))
publication_names = []
links = []
for i in range(len(divsList)):
  publication_names.append(divsList[i][0].span.text)
  links.append('https://www.scimagojr.com/'+divsList[i][0].get('href'))

In [5]:
H_index=[]
for link in links:
  if requests.get(link).status_code==200: 
      divs1 = BeautifulSoup(requests.get(link).content, 'html.parser').select("div.journaldescription.colblock table tbody tr td.hindex div.hindexnumber")
      H_index.append(divs1[0].text)

In [6]:
SJR_score=[]
for link in links:
    if requests.get(link).status_code==200: 
        divs1 = BeautifulSoup(requests.get(link).content, 'html.parser').select("div.cell1x1.dynamiccell div.cellcontent div.cellslide table tbody tr td")
        for i in range(len(divs1)):
            if(divs1[i].text == '2018'):
                SJR_score.append(divs1[i+1].text)
                break

In [18]:
df_pub = pd.DataFrame(list(zip(publication_names, H_index, SJR_score)), columns = ['publication_names', 'H_index' , 'SJR_score'])
publication_df = pd.read_csv ('SCORE_csv.csv')
publication_df['H_index'] = ""
publication_df['SJR_score'] = ""
for index, row in df_pub.iterrows():
  publication_df.loc[publication_df['publication_CR'] == row['publication_names'], ['H_index', 'SJR_score'] ] = row['H_index'],row['SJR_score']


In [21]:
publication_df[['publication_CR','H_index','SJR_score']]

,publication_CR,H_index,SJR_score
0,Journal of Experimental Social Psychology,125,2.165
1,Journal of Experimental Social Psychology,125,2.165
2,Journal of Experimental Social Psychology,125,2.165
3,Journal of Experimental Social Psychology,125,2.165
4,Journal of Experimental Social Psychology,125,2.165
5,Journal of Experimental Social Psychology,125,2.165
6,Journal of Experimental Social Psychology,125,2.165
7,Journal of Experimental Social Psychology,125,2.165
8,Journal of Experimental Social Psychology,125,2.165
9,Journal of Experimental Social Psychology,125,2.165


In [22]:
publication_df.to_csv('FINAL_SCORES.csv')